<h1 align="center"> Лабораторна робота №4 </h1>
<h2 align="center"> <b> Вивчення криптосистеми RSA та алгоритму електронного підпису; ознайомлення з методами генерації параметрів для асиметричних криптосистем </b> </h2>
<hr> <h3 align="center">Виконав студент групи ФБ-35 Куб'юк Дмитро </h3>

Мета роботи <br>
Ознайомлення з тестами перевірки чисел на простоту і методами генерації ключів для асиметричної криптосистеми типу RSA; практичне ознайомлення з системою захисту інформації на основі криптосхеми RSA, організація з використанням цієї системи засекреченого зв'язку й електронного підпису, вивчення протоколу розсилання ключів.

In [ ]:
import random
import hashlib

В якості тесту перевірки на простоту рекомендовано використовувати тест Міллера-Рабіна із попередніми пробними діленнями. Тести необхідно реалізовувати власноруч, використання готових реалізацій тестів не дозволяється.

In [477]:
def is_prime(n, k=5):
    if n <= 1:
        return False
    if n == 2 or n == 3:
        return True
    if n % 2 == 0:
        return False

    d = n - 1
    r = 0
    while d % 2 == 0:
        d //= 2
        r += 1

    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

Написати функцію пошуку випадкового простого числа з заданого інтервалу або заданої довжини, використовуючи датчик випадкових чисел та тести перевірки на простоту. В якості датчика  випадкових чисел використовуйте вбудований генератор псевдовипадкових чисел вашої мови програмування.

In [478]:
def find_random_prime_by_bits(bits):
    low = 2 ** (bits - 1)
    high = 2 ** bits - 1

    while True:
        num = random.randint(low, high)
        if is_prime(num):
            return num

За допомогою цієї функції згенерувати дві пари простих чисел p1, q1 і p2, q2 довжини щонайменше 256 біт. При цьому пари чисел беруться так, щоб p1q1 <= p2q2.

In [479]:
bits = 256

(p1, q1) = find_random_prime_by_bits(bits), find_random_prime_by_bits(bits)
(p2, q2) = find_random_prime_by_bits(bits), find_random_prime_by_bits(bits)

while p1 * q1 > p2 * q2:
    p2 = find_random_prime_by_bits(bits)
    q2 = find_random_prime_by_bits(bits)

print(f"Пара для абонента A: p1 = {p1}, q1 = {q1}")
print(f"Пара для абонента B: p2 = {p2}, q2 = {q2}")

Пара для абонента A: p1 = 85927289199903700965263832145592183202192166277492860188112680731300034543983, q1 = 88186331243732637644467378375719042813253863558702685785638861016112129626931
Пара для абонента B: p2 = 106917160949816369795055904307972826092777721725863882012775198908873748797157, q2 = 71648841097863575934875144569091688035057909860253549023185847488351577080101


Напишіть функцію для генерації ключових пар RSA. Після генерації функція повинна повертати та/або зберігати секретний ключ у вигляді пари (q, p, d) та відкритий ключ у вигляді пари (n, e). За допомогою цієї функції побудуйте схеми RSA для абонентів A та B, тобто створіть та збережіть відкриті ключі (n1, e1) і (n2, e2), а також секретні ключі d1 і d2 для подальшого використання.

In [480]:
def egcd(a, b):
    if b == 0:
        return a, 1, 0
    g, x1, y1 = egcd(b, a % b)
    return g, y1, x1 - (a // b) * y1

def modinv(a, n):
    g, x, y = egcd(a, n)
    if g != 1:
        return None
    return x % n

In [481]:
def generate_pair(p, q):
    n = p * q
    phi_n = (p - 1) * (q - 1)
    e = 65537
    d = modinv(e, phi_n)

    return (e, n), (d, n)

public_key_A, private_key_A = generate_pair(p1, q1)
public_key_B, private_key_B = generate_pair(p2, q2)

Написати програму шифрування, розшифрування і створення повідомлення з цифровим підписом для абонентів А і B. Кожна з операцій (шифрування, розшифрування, створення цифрового підпису,  перевірка цифрового підпису) повинна бути реалізована окремою процедурою, на вхід до якої повинні подаватись лише ті ключові дані, які необхідні для її виконання. 

In [482]:
def text_to_int(text):
    return int.from_bytes(text.encode(), "big")

def int_to_text(number):
    return number.to_bytes((number.bit_length() + 7) // 8, "big").decode(errors="ignore")

In [483]:
def encrypt(message, pub_key):
    e, n = pub_key
    return pow(message, e, n)

def decrypt(ciphertext, priv_key):
    d, n = priv_key
    return pow(ciphertext, d, n)

def sign(text, priv_key):
    d, n = priv_key
    h = int.from_bytes(hashlib.sha256(text.encode()).digest(), "big")
    return pow(h, d, n)

def verify(text, sig, pub_key):
    e, n = pub_key
    h = int.from_bytes(hashlib.sha256(text.encode()).digest(), "big")
    return pow(sig, e, n) == h

За допомогою датчика випадкових чисел вибрати відкрите повідомлення M і знайти криптограму для абонентів А и B, перевірити правильність розшифрування. Скласти для А і B повідомлення з цифровим підписом і перевірити його. 

In [484]:
text = ''.join(random.choices("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789", k=20))
print(text)

text_int = text_to_int(text)
result = False

ciphertext_A = encrypt(text_int, public_key_A)
ciphertext_B = encrypt(text_int, public_key_B)

decrypted_A = int_to_text(decrypt(ciphertext_A, private_key_A))
decrypted_B = int_to_text(decrypt(ciphertext_B, private_key_B))

if (text == decrypted_A) and (text == decrypted_B):
    result = True

signature_A = sign(text, private_key_A)
signature_B = sign(text, private_key_B)

result &= verify(text, signature_A, public_key_A)
result &= verify(text, signature_B, public_key_B)

print(result)


Pbx4BR4ZiIWG2TltYjN0
True


За допомогою раніше написаних на попередніх етапах програм організувати роботу протоколу конфіденційного розсилання ключів з підтвердженням справжності по відкритому каналу за допомогою алгоритму RSA. Протоколи роботи кожного учасника (відправника та приймаючого) повинні бути реалізовані у вигляді окремих процедур, на вхід до яких повинні подаватись лише ті ключові дані, які необхідні для виконання. Перевірити роботу програм для випадково обраного ключа 0 < k < n. 
Кожна з наведених операцій повинна бути реалізована у вигляді окремої процедури, інтерфейс якої повинен приймати лише ті дані, які необхідні для її роботи; наприклад, функція Encrypt(), яка шифрує повідомлення для абонента, повинна приймати на вхід повідомлення та відкритий ключ адресата (і тільки його), повертаючи в якості результату шифротекст. Відповідно, програмний код повинен містити сім високорівневих процедур: GenerateKeyPair(), Encrypt(), Decrypt(), Sign(), Verify(), SendKey(), ReceiveKey().

In [485]:
def send_key(sender_priv, receiver_pub, k):
    enc_k = encrypt(k, receiver_pub)
    signature = sign(str(enc_k), sender_priv)
    return enc_k, signature

def receive_key(receiver_priv, sender_pub, enc_k, signature):
    if not verify(str(enc_k), signature, sender_pub):
        print("something went wrong")

    k = decrypt(enc_k, receiver_priv)
    return k

In [486]:
eB, nB = public_key_B
k = random.randint(1, nB - 1)

ciphertext, signature = send_key(private_key_A, public_key_B, k)
received_k = receive_key(private_key_B, public_key_A, ciphertext, signature)

print("Оригінальний ключ k:", k)
print("Отриманий ключ:     ", received_k)
print("Підпис пройшов:", received_k == k)

Оригінальний ключ k: 1492170601270032944928206351824003318645571963335690993040216054404778757302341068196003398907337011694717788137274187164925282802061049446189254508531101
Отриманий ключ:      1492170601270032944928206351824003318645571963335690993040216054404778757302341068196003398907337011694717788137274187164925282802061049446189254508531101
Підпис пройшов: True


Кожну операцію рекомендується перевіряти шляхом взаємодії із тестовим середовищем, розташованим за адресою http://asymcryptwebservice.appspot.com/?section=rsa. 

Зашифрування/розшифрування <br>
![alt text](data/decryption.png "Title")

In [487]:
text = "ABOBA2281337"
text_int = text_to_int(text)

p, q = find_random_prime_by_bits(bits), find_random_prime_by_bits(bits)
pub, priv = generate_pair(p, q)

print(f"Публічний ключ (e, n): {pub}\nПриватний ключ (d, n): {priv}\nВідкритий текст: {text_int}\nЗашифрований текст: {encrypt(text_int, pub)}")

Публічний ключ (e, n): (65537, 7074231315609210811162596426629290554863034824642465927390499042861963257736716491261037079620861135238759588899186617410657928737580754748386505493041689)
Приватний ключ (d, n): (1050281073300236830990311781605856189615291039317808161397524386026929864012337478415488415526637545676576540882424249325412062000895707549427901568399433, 7074231315609210811162596426629290554863034824642465927390499042861963257736716491261037079620861135238759588899186617410657928737580754748386505493041689)
Відкритий текст: 20196689031617123137669509943
Зашифрований текст: 5606739529863478092663997218560206333011080192635517763918468704040096284803080753588742250695807095455778151697503435213444785016625431902523918280667212
